# Prepare Zondo Commission articles

## Import required libraries and setup query

In [1]:
import pandas as pd
import pandas_gbq
from sklearn.model_selection import train_test_split

In [2]:
def import_article_data(sample: bool = False) -> pd.DataFrame:
    '''Get Id, Title, Synopsis and CleanBody for relevant articles'''
    
    if sample:
        main_table = '`cas-lake.zondo_kg.stratified_sample_30`'
    else: 
        main_table = '`cas-lake.zondo_kg.raw_article_data`'
    query_or_table = f'''
        SELECT
            a.Id
            , a.SampleType
            , a.Title
            , a.Permatitle
            , a.Synopsis
            , b.CleanBody
        FROM {main_table} a
        INNER JOIN `cas-lake.zondo_kg.clean_body` b
        ON a.Id = b.Id
        WHERE Included = TRUE
        '''

    df = pandas_gbq.read_gbq(project_id='cas-lake', query_or_table=query_or_table, use_bqstorage_api=True)

    return df

## Get sample data

In [3]:
sample_df = import_article_data(sample = True)
sample_df.head(2)

Downloading: 100%|██████████|


,Id,SampleType,Title,Permatitle,Synopsis,CleanBody
0,ed94c34a-8499-44f9-afb4-f8df96bb8843,general,Zondo commission to issue a summons for Jacob ...,zondo-commission-to-issue-a-summons-for-jacob-...,The state capture commission of inquiry says i...,The state capture commission of inquiry says i...
1,b98bba34-c5d7-440b-b7a5-e365fabf4bc3,general,WATCH LIVE | State capture inquiry continues t...,watch-live-state-capture-inquiry-continues-to-...,The state capture inquiry will on Friday conti...,PwC South Africa auditor Pule Mothibe admitted...


In [4]:
# Check how many articles we have per type overall
sample_df.groupby('SampleType').count()[['Id']]

,Id
SampleType,
analysis,2
general,22
opinion,6


In [5]:
# Collate the 3 text elements into a new field AllText
sample_df['AllText'] = sample_df['Title'] + '\n' + sample_df['Synopsis'] + '\n' + sample_df['CleanBody']

# Perform the train/test split - 10 articles will be used to investigate what improvements
# can be made to the model outputs and develop additional methods for realising these
# improvements, the remaining 20 articles will be used to test if improvements are present
# in the unseen data of the test set
train, test = train_test_split(sample_df, test_size=20, stratify=sample_df['SampleType'])

In [6]:
# Check how many articles we have per type in the train set
train.groupby('SampleType').count()[['Id']]

,Id
SampleType,
analysis,1
general,7
opinion,2


In [7]:
# Check how many articles we have per type in the test set
test.groupby('SampleType').count()[['Id']]

,Id
SampleType,
analysis,1
general,15
opinion,4


In [8]:
# Add a 'split' column with default value 'train'
sample_df['Split'] = 'train'
# Set the articles from 'test':
sample_df.loc[sample_df['Id'].isin(test['Id']), 'Split'] = 'test'

In [9]:
sample_df[['Id', 'Permatitle', 'SampleType', 'AllText', 'Split']].to_parquet('source_data/sample_text_30.pq')

## Get full data

In [10]:
full_df = import_article_data(sample = False)

Downloading: 100%|██████████|


In [11]:
# Check how many articles we have per type overall
full_df.groupby('SampleType').count()[['Id']]

,Id
SampleType,
analysis,142
general,1551
opinion,388


In [12]:
# Collate the 3 text elements into a new field AllText
full_df['AllText'] = full_df['Title'] + '\n' + full_df['Synopsis'] + '\n' + full_df['CleanBody']

# Adding in a cleaning step because some Synopsis entries were found to have the strong tag
full_df['AllText'] = full_df['AllText'].str.replace('<strong>|</strong>', '', regex=True)

In [13]:
full_df[['Id', 'Permatitle', 'SampleType', 'AllText']].to_parquet('source_data/full_text.pq')